# Model using Neural Network

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [2]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [3]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [4]:
from sklearn.utils import shuffle

shuffled_df = shuffle(df, random_state = 42)

## Split Data for Training and Validation


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state=42)

## The training and validation set size

In [6]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181


## Function Transform Dataframe into Tensor Dataset

In [7]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds

## Transform Training and Validation Dataframe into Tensor Dataset

In [8]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [9]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [10]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [11]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [12]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

In [13]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Encode columns based on their data type

In [14]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [15]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [18]:
x = keras.layers.Dense(64, activation = 'relu')(all_features)
x = keras.layers.Dense(32, activation = 'relu')(x)
x = keras.layers.Dense(16, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [19]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [20]:
from IPython.core.display import display, HTML

# Enable horizontal scrolling for output
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Generate a long output
long_output = "A" * 1000

model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 6s 31ms/step - loss: 0.6995 - accuracy: 0.5284 - val_loss: 0.6912 - val_accuracy: 0.5249
Epoch 2/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6998 - accuracy: 0.5118 - val_loss: 0.6915 - val_accuracy: 0.5470
Epoch 3/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6897 - accuracy: 0.5256 - val_loss: 0.6907 - val_accuracy: 0.5414
Epoch 4/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6867 - accuracy: 0.5422 - val_loss: 0.6916 - val_accuracy: 0.5028
Epoch 5/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6875 - accuracy: 0.5450 - val_loss: 0.6913 - val_accuracy: 0.5138
Epoch 6/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.5408 - val_loss: 0.6928 - val_accuracy: 0.4530
Epoch 7/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6830 - accuracy: 0.5588 - val_loss: 0.6927 - val_accuracy: 0.4862
Epoch 8/300


Epoch 59/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6300 - accuracy: 0.6362 - val_loss: 0.7174 - val_accuracy: 0.5083
Epoch 60/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6270 - accuracy: 0.6432 - val_loss: 0.7118 - val_accuracy: 0.5138
Epoch 61/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6256 - accuracy: 0.6445 - val_loss: 0.7127 - val_accuracy: 0.5414
Epoch 62/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6378 - accuracy: 0.6293 - val_loss: 0.7133 - val_accuracy: 0.5083
Epoch 63/300
37/37 [==============================] - 0s 6ms/step - loss: 0.6122 - accuracy: 0.6598 - val_loss: 0.7168 - val_accuracy: 0.5249
Epoch 64/300
37/37 [==============================] - 0s 7ms/step - loss: 0.6217 - accuracy: 0.6487 - val_loss: 0.7298 - val_accuracy: 0.5138
Epoch 65/300
37/37 [==============================] - 0s 8ms/step - loss: 0.6285 - accuracy: 0.6155 - val_loss: 0.7249 - val_accuracy: 0.5083
Epoch 

37/37 [==============================] - 0s 7ms/step - loss: 0.5431 - accuracy: 0.7178 - val_loss: 0.8264 - val_accuracy: 0.4972
Epoch 117/300
37/37 [==============================] - 0s 8ms/step - loss: 0.5717 - accuracy: 0.7234 - val_loss: 0.7861 - val_accuracy: 0.5083
Epoch 118/300
37/37 [==============================] - 0s 8ms/step - loss: 0.5598 - accuracy: 0.7109 - val_loss: 0.8013 - val_accuracy: 0.4972
Epoch 119/300
37/37 [==============================] - 0s 8ms/step - loss: 0.5613 - accuracy: 0.7192 - val_loss: 0.8460 - val_accuracy: 0.4917
Epoch 120/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5553 - accuracy: 0.7068 - val_loss: 0.8071 - val_accuracy: 0.4807
Epoch 121/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5515 - accuracy: 0.7220 - val_loss: 0.8053 - val_accuracy: 0.4972
Epoch 122/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5441 - accuracy: 0.7068 - val_loss: 0.8082 - val_accuracy: 0.5249
Epoch 123/300

37/37 [==============================] - 0s 7ms/step - loss: 0.4815 - accuracy: 0.7759 - val_loss: 0.9062 - val_accuracy: 0.5193
Epoch 174/300
37/37 [==============================] - 0s 6ms/step - loss: 0.5125 - accuracy: 0.7441 - val_loss: 0.9204 - val_accuracy: 0.5193
Epoch 175/300
37/37 [==============================] - 0s 8ms/step - loss: 0.4915 - accuracy: 0.7663 - val_loss: 0.9705 - val_accuracy: 0.5525
Epoch 176/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5133 - accuracy: 0.7400 - val_loss: 0.9234 - val_accuracy: 0.5359
Epoch 177/300
37/37 [==============================] - 0s 7ms/step - loss: 0.4938 - accuracy: 0.7427 - val_loss: 0.9101 - val_accuracy: 0.5249
Epoch 178/300
37/37 [==============================] - 0s 7ms/step - loss: 0.5040 - accuracy: 0.7510 - val_loss: 0.9327 - val_accuracy: 0.5414
Epoch 179/300
37/37 [==============================] - 0s 7ms/step - loss: 0.4752 - accuracy: 0.7552 - val_loss: 0.9953 - val_accuracy: 0.5359
Epoch 180/300

37/37 [==============================] - 0s 7ms/step - loss: 0.4297 - accuracy: 0.7898 - val_loss: 1.3647 - val_accuracy: 0.4917
Epoch 231/300
37/37 [==============================] - 0s 7ms/step - loss: 0.4359 - accuracy: 0.7856 - val_loss: 1.2191 - val_accuracy: 0.5083
Epoch 232/300
37/37 [==============================] - 0s 6ms/step - loss: 0.4073 - accuracy: 0.8022 - val_loss: 1.2863 - val_accuracy: 0.5138
Epoch 233/300
37/37 [==============================] - 0s 6ms/step - loss: 0.4284 - accuracy: 0.7911 - val_loss: 1.1968 - val_accuracy: 0.5193
Epoch 234/300
37/37 [==============================] - 0s 7ms/step - loss: 0.4025 - accuracy: 0.8133 - val_loss: 1.1804 - val_accuracy: 0.5083
Epoch 235/300
37/37 [==============================] - 0s 7ms/step - loss: 0.4255 - accuracy: 0.7870 - val_loss: 1.2565 - val_accuracy: 0.5359
Epoch 236/300
37/37 [==============================] - 0s 7ms/step - loss: 0.4333 - accuracy: 0.8050 - val_loss: 1.1538 - val_accuracy: 0.5304
Epoch 237/300

37/37 [==============================] - 0s 8ms/step - loss: 0.3720 - accuracy: 0.8133 - val_loss: 1.4775 - val_accuracy: 0.5249
Epoch 288/300
37/37 [==============================] - 0s 7ms/step - loss: 0.3388 - accuracy: 0.8285 - val_loss: 1.6898 - val_accuracy: 0.5138
Epoch 289/300
37/37 [==============================] - 0s 7ms/step - loss: 0.3649 - accuracy: 0.8188 - val_loss: 1.5734 - val_accuracy: 0.5028
Epoch 290/300
37/37 [==============================] - 0s 7ms/step - loss: 0.3817 - accuracy: 0.8437 - val_loss: 1.6463 - val_accuracy: 0.5691
Epoch 291/300
37/37 [==============================] - 0s 7ms/step - loss: 0.3397 - accuracy: 0.8396 - val_loss: 1.6034 - val_accuracy: 0.5470
Epoch 292/300
37/37 [==============================] - 0s 7ms/step - loss: 0.3527 - accuracy: 0.8340 - val_loss: 1.5310 - val_accuracy: 0.5193
Epoch 293/300
37/37 [==============================] - 0s 7ms/step - loss: 0.3415 - accuracy: 0.8423 - val_loss: 1.6436 - val_accuracy: 0.5193
Epoch 294/300

In [ ]:
df.info()

In [69]:
input_data = {
    'gender' : 1,
    'age'    :21,
    'workout_experience' : 0,
    'workout_time' : 60,
    'weight' : 65,
    'height' : 170,
}

In [70]:
input_dict = {}

In [71]:
for feature, value in input_data.items() :
    input_dict[feature] = tf.convert_to_tensor([value])

In [72]:
input_dict

{'gender': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>,
 'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([21])>,
 'workout_experience': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 'workout_time': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([60])>,
 'weight': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([65])>,
 'height': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([170])>}

In [76]:
predictions = model.predict(input_dict)

1/1 [==============================] - 0s 34ms/step


In [77]:
predictions

array([[0.5033811]], dtype=float32)

In [78]:
if predictions[0][0] >= 0.5 :
    print("You should do muscle up !")
else :
    print("You should do weight loss !")

You should do muscle up !


In [21]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'model_1_v2.pkl')

INFO:tensorflow:Assets written to: ram://a02c0d99-312e-4275-b03e-2019695547c0/assets


['model_1_v2.pkl']